# Inhalation aligned LFP rasters

In [1]:
import numpy as np
from utils import *
from tqdm import tqdm
import os

In [ ]:
# defining directories
lfp_dir = r"E:\clickbait-ephys\data\preprocessed"
sniff_dir = r"E:\clickbait-ephys\data\sniff"
save_dir = r"E:\clickbait-ephys\figures\lfp_sniff_dark"


# defining parameters
mice = ['6000', '6001', '6002', '6003']
mice = ['6002']
sessions = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
sessions = ['0', '1']
fs = 30_000  # Define sampling rate for neural data
sfs = 1_000  # Define sampling rate for sniff
lfs = 1_000  # Define sampling rate for lfps


for mouse in mice:
    spike_sessions = [s for s in os.listdir(os.path.join(lfp_dir, mouse)) 
                  if os.path.isdir(os.path.join(lfp_dir, mouse, s))]
    sniff_sessions = [s for s in os.listdir(os.path.join(sniff_dir, mouse)) 
                  if os.path.isdir(os.path.join(sniff_dir, mouse, s))]

    # loop through sessions
    common_sessions = list(set(set(spike_sessions).intersection(sniff_sessions).intersection(sessions)))
    for session in common_sessions:

        print(f"\nProcessing {mouse}/{session}")
        # load the sniff and lfp data
        sniff_params_file = os.path.join(sniff_dir, mouse, session, 'sniff_params.mat')
        lfp_file = os.path.join(lfp_dir, mouse, session, 'lfp.npy')
        if not os.path.exists(sniff_params_file) or not os.path.exists(lfp_file):
            print(f"Missing files for {mouse}/{session}. Skipping...")
            continue
        inh_start, _, inh_end, _ = load_sniff_MATLAB(sniff_params_file)
        lfps = np.load(lfp_file)

        # downsample the lfp data
        lfps = signal.decimate(lfps, fs // sfs, zero_phase=True)

        # cleaning the inhalation times
        freqs = sfs / np.diff(inh_start)
        remove_indicies = np.where((freqs < 2) | (freqs > 12))[0]
        inh_start = np.delete(inh_start, remove_indicies)
        inh_end = np.delete(inh_end, remove_indicies)

        # Creating the sniff rasters
        current_save_path = os.path.join(save_dir, mouse, session)
        os.makedirs(current_save_path, exist_ok=True)
        print("Building sniff rasters...")
        build_sniff_rasters(lfps, inh_start, inh_end, current_save_path, max_workers = 4, dark_mode = True)


### Building PSDs

In [ ]:
# define the data directory
data_dir = r"E:\clickbait-ephys\data\preprocessed"
psd_dir = r"E:\clickbait-ephys\data\psd_noise"
individual_psd_dir = r"E:\clickbait-ephys\figures\individual_psd_linearx_noiseremoved_sensitivityinf"
os.makedirs(psd_dir, exist_ok=True)
os.makedirs(individual_psd_dir, exist_ok=True)

# Define sampling rate for lfps
lfs = 1000

# Define the mice, sessions and regions of interest
mice = ['6000', '6001', '6002', '6003']
sessions = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
regions = ['ob', 'hc']

# Define the parameters for PSD computation
nperseg = 4000 # length of each segment
noverlap = 2000 # number of overlapping points
nfft = 4096 # number of points in FFT
sensitivity = np.inf # sensitivity for removing noisy channels

# Initialize dictionaries to PSD data
data_dict = {regions[0]: [], regions[1]: []}

# Looping through the mice and sessions
files = os.listdir(data_dir)
for mouse in mice:
    print(f'\nProcessing Mouse: {mouse}')
    common_sessions = list(set(os.listdir(os.path.join(data_dir, mouse))).intersection(set(sessions)))

    for session in tqdm(common_sessions, desc='Sessions'):
        lfp_file = os.path.join(data_dir, mouse, session, 'lfp.npy')
        if not os.path.exists(lfp_file):
            print(f"Missing LFP file for {mouse}/{session}. Skipping...")
            continue

        # loading the lfp data
        lfp = np.load(lfp_file)

        for region in regions:
            if region == 'ob':
                current_lfp = lfp[16:]
            elif region == 'hc':
                current_lfp = lfp[:16]

            # Check if the current_lfp is empty
            if current_lfp.size == 0:
                print(f"Warning: Empty LFP data for {mouse}/{session}/{region}. Skipping...")
                continue

            # ensure the lfp is long enough
            if current_lfp.shape[1] < 100 * lfs:
                print(f"Warning: LFP data too short for {mouse}/{session}/{region}. Skipping...")
                continue

            # Create the directory for saving PSD plots
            current_plot_dir = os.path.join(individual_psd_dir, mouse)
            os.makedirs(current_plot_dir, exist_ok=True)

            # Compute the power spectral density in each channel and average the results across channels
            freqs, psd, noisy_channels = compute_multichannel_psd_noiseremoved(current_lfp, lfs, nperseg = nperseg, noverlap = noverlap, nfft = nfft, method = 'mean_psd', plot_dir = current_plot_dir + f'/{session}_{region}_psd.png', region=region, sensitivity = sensitivity)
            if psd is None:
                continue
            
            # append the results to the data dictionary
            data_dict[region].append({
                'mouse': mouse,
                'session': session,
                'freqs': freqs,
                'psd': psd,
                'noisey_channels': noisy_channels
            })

def convert_psd_dict_to_df(psd_list):
    """ Converts the list of PSD dictionaries to a Pandas DataFrame """
    data_list = []
    for entry in psd_list:
        mouse = entry['mouse']
        session = entry['session']
        freqs = entry['freqs']
        psd = entry['psd']

        # Create a dataframe for each session
        temp_df = pd.DataFrame({'Frequency': freqs, 'psd': psd})
        temp_df['Mouse'] = mouse
        temp_df['Session'] = session
        data_list.append(temp_df)

    # Concatenate all dataframes
    return pd.concat(data_list, ignore_index=True)

# Convert both OB and HC data to DataFrames
ob_df = convert_psd_dict_to_df(data_dict['ob'])
hc_df = convert_psd_dict_to_df(data_dict['hc'])

# Save the DataFrames to CSV files
ob_df.to_csv(os.path.join(psd_dir, 'ob_psd.csv'), index=False)
hc_df.to_csv(os.path.join(psd_dir, 'hc_psd.csv'), index=False)

In [3]:

save_dir = r"E:\clickbait-ephys\figures\psd_dark_noise"
os.makedirs(save_dir, exist_ok=True)
psd_dir = r"E:\clickbait-ephys\data\psd_noise"
ob_df = pd.read_csv(os.path.join(psd_dir, 'ob_psd.csv'))
hc_df = pd.read_csv(os.path.join(psd_dir, 'hc_psd.csv'))
# plotting the results
plot_PSD(ob_df, hc_df, save_dir, dark_mode=True)